In [38]:
import pandas as pd

In [39]:
import os
from sklearn import preprocessing

In [40]:
from collections import deque

In [41]:
import random
import numpy as np

In [42]:
import time 

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [44]:
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [45]:
seq_len =60
future_period_predict=3
ratio_to_predict='LTC-USD'
epochs = 10
batch_size = 64
name = f'{seq_len}-SEQ-{future_period_predict}-PRED-{int(time.time())}'

In [46]:
df = pd.read_csv('data/ltc-usd.csv',names = ['time','low','high','open','close','voolume'])

In [47]:
df

,time,low,high,open,close,voolume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978
...,...,...,...,...,...,...
101878,1535215020,58.009998,58.020000,58.020000,58.009998,7.301921
101879,1535215080,58.009998,58.020000,58.009998,58.020000,23.802017
101880,1535215140,58.020000,58.020000,58.020000,58.020000,6.953497
101881,1535215200,58.020000,58.080002,58.020000,58.080002,202.403183


In [48]:
main_df = pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider
for ratio in ratios:  # begin iteration
    dataset = f'data/{ratio}.csv'  # get the full path to the file.
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)


In [49]:
main_df  # how did we do??

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume
time,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.010010,26.019083
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.000000,8.449400
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.750000,26.994646
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.000000,77.355759
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.000000,7.503300
...,...,...,...,...,...,...,...,...
1535214960,6713.140137,0.769891,58.020000,6.434783,531.479980,1.208560,279.359985,11.280577
1535215020,6714.520020,1.002652,58.009998,7.301921,531.479980,0.016868,279.359985,8.790519
1535215080,6714.520020,1.021925,58.020000,23.802017,531.469971,0.013854,279.369995,1.311763


In [50]:
main_df['future'] = main_df[f'{ratio_to_predict}_close'].shift(-future_period_predict)

In [51]:
main_df.head(20)

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume,future
time,,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.010010,26.019083,96.389999
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.000000,8.449400,96.519997
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.750000,26.994646,96.440002
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.000000,77.355759,96.470001
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.000000,7.503300,96.400002
1528969020,6477.220215,2.731950,96.440002,95.524078,869.450012,0.865200,485.989990,85.877251,96.400002
1528969080,6480.000000,2.174240,96.470001,175.205307,869.989990,23.534929,485.989990,160.915192,96.400002
1528969140,6479.990234,0.903100,96.400002,43.652802,870.000000,2.300000,485.989990,61.371887,96.400002
1528969200,6478.660156,3.258786,96.400002,8.160000,870.320007,9.255514,485.989990,42.687656,96.400002


In [52]:
def classify(current,future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [53]:
def preprocess_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col != 'target':
            #get the percentage change from one row to next row, drop the first row since it will be NA
            df[col] = df[col].pct_change()
            # scale the values for more accurate calculation, value now between 0-1
            df[col] = preprocessing.scale(df[col].values)
    # just in case
    df.dropna(inplace=True)
    
    sequential_data = []
    # use deque to create an empty list with always length = seq_len (60)
    prev_days = deque(maxlen=seq_len)
    
    #iterate over the columns, df.value contains no more time, only the values(price, volumn,target)
    for i in df.values:
        # -1 meaning not appending the target
        prev_days.append(i[:-1])
        if len(prev_days) == seq_len:
            #append(x ,y) x=features, y=labels (hoping to predict a 0 or 1)
            sequential_data.append([np.array(prev_days), i[-1]])
            
    random.shuffle(sequential_data)
    
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target == 1:
            buys.append([seq,target])

    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys),len(sells))
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys+sells
    random.shuffle(sequential_data)
    
    X = []
    Y = []
    
    for seq, target in sequential_data:
        X.append(seq)
        Y.append(target)
        
    return np.array(X), Y
        

In [54]:
main_df['target']=list(map(classify,main_df[f'{ratio_to_predict}_close'],main_df['future']))

In [55]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

In [56]:
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df            = main_df[(main_df.index < last_5pct)]

In [57]:
main_df

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume,future,target
time,,,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.010010,26.019083,96.389999,0
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.000000,8.449400,96.519997,0
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.750000,26.994646,96.440002,0
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.000000,77.355759,96.470001,1
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.000000,7.503300,96.400002,0
...,...,...,...,...,...,...,...,...,...,...
1534921800,6686.250000,0.478039,57.509998,18.782650,551.299988,0.336000,285.230011,38.141129,57.509998,0
1534921860,6686.250000,0.440793,57.500000,8.449425,551.299988,0.010847,285.489990,17.549879,57.509998,1
1534921920,6686.250000,2.678847,57.509998,6.070000,551.299988,5.713912,285.739990,6.953944,57.509998,0


In [58]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [59]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 77926 validation: 3868
Dont buys: 38963, buys: 38963
VALIDATION Dont buys: 1934, buys: 1934


In [60]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [61]:
model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [62]:
model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [63]:
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

In [64]:
model.add(Dense(2, activation='softmax'))

In [65]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay =1e-6)

In [66]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = opt,
             metrics = ['accuracy'])

In [81]:
tensorboard = TensorBoard(log_dir=f'log\{NAME}')

In [82]:
NAME = f"{ratio_to_predict}-{seq_len}-SEQ-{future_period_predict}-PRED-{int(time.time())}"

In [86]:
filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}.hd5"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models\{}.model".format(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

In [87]:
train_y = np.asarray(train_y)
validation_y = np.asarray(validation_y)

In [88]:
history = model.fit(
    train_x, train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint]
)

Train on 77926 samples, validate on 3868 samples
Epoch 1/10
77824/77926 [============================>.] - ETA: 0s - loss: 0.6846 - accuracy: 0.5602

ValueError: Layer has 2 states but was passed 0 initial states.